In [147]:
import datetime as dt
import math
import json
from pathlib import Path
import pickle
import time

from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import httpx

# for i in range(10):
#     clear_output(wait=True)
#     print(i)
#     time.sleep(0.1)

DATA_DIR = Path('./data')
WORKS_DATA_DIR = DATA_DIR / 'works_of_institutions'
AUTHORS_DIR = DATA_DIR / 'works_by_author'
print(len(list(WORKS_DATA_DIR.glob('*.json'))))

1134


## institutional works to author_id

In [28]:
work_file = WORKS_DATA_DIR / 'work-2020-true-1.json'

with work_file.open('r', encoding='UTF-8') as infile:
    work_json = json.load(infile)
print(len(work_json['results']))

print(work_json['results'][0].keys())
for r in work_json['results']:
    clear_output(wait=True)
    print(r['id'])
    matched_authors = [x for x in r['authorships'] if any(i for i in x['institutions'] if i['id'] in SAMPLE_INSTITUTIONS)]
    for auth in matched_authors:
        print(auth['author']['id'], auth['author']['display_name'], [i['display_name'] for i in auth['institutions']])
    print()
    


https://openalex.org/W3087139986
https://openalex.org/A2814149936 Yuzi Tian ['University of Michigan–Ann Arbor', 'Department of Rheumatology and Immunology, Xiangya Hospital, Central South University, Changsha, Hunan, China.']
https://openalex.org/A2607418774 Qiufang Deng ['University of Michigan–Ann Arbor', 'Department of Rheumatology and Immunology, Xiangya Hospital, Central South University, Changsha, Hunan, China.']
https://openalex.org/A2274029971 Hasan B. Alam ['University of Michigan–Ann Arbor']
https://openalex.org/A2107458222 Yongqing Li ['University of Michigan–Ann Arbor']



In [13]:
SAMPLE_INSTITUTIONS = (
    'https://openalex.org/I27837315',
    'https://openalex.org/I4210114445',
    'https://openalex.org/I4210151884'
)

In [ ]:
author_ids = set()

for work_file in WORKS_DATA_DIR.glob("work-*.json"):
    clear_output(wait=True)
    print(work_file)
    with work_file.open('r', encoding='UTF-8') as infile:
        work_json = json.load(infile)
    for r in work_json['results']:
        print(r['id'])
        matched_authors = [x for x in r['authorships'] if any(i for i in x['institutions'] if i.get('id') in SAMPLE_INSTITUTIONS)]
        for auth in matched_authors:
            author_id = auth['author'].get('id')
            if author_id and author_id not in author_ids:
                author_ids.add(auth['author']['id'])
                print(author_id, auth['author']['display_name'], [i.get('display_name') for i in auth['institutions']])
    print('up to', len(author_ids))
print('done')

In [38]:
authors_out = WORKS_DATA_DIR / 'authors_from_works_of_institutions.json'
with authors_out.open('w', encoding='UTF-8') as outfile:
    json.dump(list(author_ids), outfile)
print(len(author_ids))

99174


### author_id to authored works

In [249]:
CONTACT_EMAIL = "matvan@umich.edu"

In [52]:
with authors_out.open('r', encoding='UTF-8') as infile:
    authors = list(json.load(infile))
len(authors)

99174

In [157]:
def sleep_until_one_second_after(earlier):
    target = earlier + dt.timedelta(seconds=1)
    while True:
        diff = (target - dt.datetime.now()).total_seconds()
        if diff <= 0:
            return
        elif diff <= 0.1:
            time.sleep(0.01)
        elif diff <= 1.5:
            time.sleep(0.1)
        else:
            time.sleep(1)

basically_now = dt.datetime.now()
print(basically_now)
time.sleep(0.8)
sleep_until_one_second_after(basically_now)
print(dt.datetime.now())

2022-08-31 10:08:50.377025
2022-08-31 10:08:51.380112


In [154]:
def strip_work(rec):
    new_rec = {'id': minimize_id(rec['id'])}
    new_rec.update(**{k: rec[k] for k in ('ids', 'publication_year', 'publication_date', 'title', 'type')})
    new_rec['author_list'] = [minimize_id(x.get('author', {}).get('id')) for x in rec['authorships']]
    new_rec['host_venue_id'] = minimize_id(rec.get('host_venue', {}).get('id'))
    new_rec['abstract'] = reconstitute_abstract(rec.get('abstract_inverted_index', {}))
    return new_rec

# with (AUTHORS_DIR / 'A648449047-001.json').open('r', encoding='UTF-8') as infile:
#     test_full_json = json.load(infile)
# print(len(test_full_json['results']))

# test_item = test_full_json['results'][0]


# stripped = strip_work(test_item)
# print(len(str(stripped)), len(str(test_item)), '\n', stripped, '\n\n', test_item)

In [155]:
def minimize_id(s):
    if not s:
        return ""
    return s.replace("https://openalex.org/", "")

test_string = 'https://openalex.org/A2297362609'
print(test_string, minimize_id(test_string))

https://openalex.org/A2297362609 A2297362609


In [171]:
def reconstitute_abstract(abstract_inverted_index):
    if not abstract_inverted_index:
        return ''
    reverted = dict()
    for k, vs in abstract_inverted_index.items():
        for v in vs:
            reverted[v] = k
    abstract = ' '.join(s for i, s in sorted(reverted.items(), key=lambda x: (x[0])))
    return abstract

# i like donuts and i like pie
test_abstract = {'and': [3], 'donuts': [2], 'i': [0, 4], 'like': [1, 5], 'pie': [6]}
print(reconstitute_abstract(test_abstract))

i like donuts and i like pie


In [167]:
last_server_hit = dt.datetime.now()

In [216]:
def count_exhausted(meta):
    return meta['per_page'] * meta['page'] >= meta['count']

In [234]:
request_timeout = 5
per_page = 200

for author_n, full_author_id in enumerate(authors):
    clear_output(wait=True)
    author_id = full_author_id.replace('https://openalex.org/', '')
    print(author_n, 'of', len(authors), author_id, i)

    base_url = f"https://api.openalex.org/works?mailto={CONTACT_EMAIL}"
    filtering = f"filter=author.id:{author_id}"
    
    for i in range(1, 1000):
        pagination = f"per_page={per_page}&page={i}"
        url = "&".join((base_url, filtering, pagination))
        print(url)

        filename = f"{author_id}-{i:0>3}.json"
        path = AUTHORS_DIR / filename
        
        # we're going to gather and handle the errors later, 
        # not retry now
        error_path = DATA_DIR / f"ERROR-{filename}"
        if error_path.exists():
            continue
        
        # if the path is there, we can just check to see
        # if we need to bother fetching the next
        if path.exists():
            with path.open('r', encoding='UTF-8') as infile:
                existing_json = json.load(infile)
            existing_meta = existing_json.get('meta')
            if count_exhausted(existing_meta):
                print(f'Skipping {author_id} based on {path}')
                break
                
        testnum += 1  
        
        if path.exists():
            print(f"Skipping {path}")
            continue

        sleep_until_one_second_after(last_server_hit)
        print(f'Going back to OpenAlex after {round((dt.datetime.now() - last_server_hit).total_seconds(), 1)} seconds')
        try:            
            last_server_hit = dt.datetime.now()
            print(last_server_hit)
            raw = httpx.get(url, timeout=request_timeout).json()
            last_server_hit = dt.datetime.now()
        except Exception as err:
            error_path.write_text("\n".join((full_author_id, url, str(err))))
            print(err)
            print(error_path)
            continue

        print(raw['meta'])
        
        if raw['results']:
            print(path)
            
            new_result = {}
            new_result['meta'] = raw['meta']
            new_result['results'] = [strip_work(w) for w in raw['results']]
            
            with path.open('w', encoding='UTF-8') as outfile:
                json.dump(new_result, outfile)
               
        # This (or an earlier) page has exhausted the record count of this author
        if count_exhausted(raw['meta']):
            break


99173 of 99174 A4274067696 1
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A4274067696&per_page=200&page=1
Going back to OpenAlex after 1.0 seconds
2022-09-01 13:03:40.827334
{'count': 1, 'db_response_time_ms': 23, 'page': 1, 'per_page': 200}
data\works_by_author\A4274067696-001.json


In [232]:
"\n".join((full_author_id, url, str('hi')))

'https://openalex.org/A2403083729\nhttps://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2403083729&per_page=200&page=1\nhi'

copying from above, sorry, but now we're going to hit the errored files

In [235]:
authors[:10]

['https://openalex.org/A2669589972',
 'https://openalex.org/A1978526575',
 'https://openalex.org/A1985405156',
 'https://openalex.org/A2098295374',
 'https://openalex.org/A1673559269',
 'https://openalex.org/A2912214977',
 'https://openalex.org/A2893840941',
 'https://openalex.org/A3140563759',
 'https://openalex.org/A2151249208',
 'https://openalex.org/A3107835262']

In [ ]:
errored_authors = []
for p in (DATA_DIR / '2022-08-31-errors').glob('ERROR*.json'):
    author_id = p.name.split('-')[1]
    full_author_id = f"https://openalex.org/{author_id}"
#     print(p, author_id, full_author_id)
    errored_authors.append(full_author_id)
errored_authors = list(set(errored_authors))

In [252]:
len(set(errored_authors))

284

In [254]:
# give lots of time for these
request_timeout = 120
# and smaller chunks
per_page = 20

for author_n, full_author_id in enumerate(list(set(errored_authors))):
    clear_output(wait=True)
    author_id = full_author_id.replace('https://openalex.org/', '')
    print(author_n, 'of', len(set(errored_authors)), author_id, i)

    base_url = f"https://api.openalex.org/works?mailto={CONTACT_EMAIL}"
    filtering = f"filter=author.id:{author_id}"
    
    for i in range(1, 1000):
        pagination = f"per_page={per_page}&page={i}"
        url = "&".join((base_url, filtering, pagination))
        print(url)

        filename = f"{author_id}-{i:0>3}.json"
        path = AUTHORS_DIR / filename
        
        # this is the error handling version!
        error_path = DATA_DIR / f"ERROR-{filename}"
        #         if error_path.exists():
        #             continue
        
        # we're going to retry & overwrite them all.
        # there's 664 errored files
        #         # if the path is there, we can just check to see
        #         # if we need to bother fetching the next
        #         if path.exists():
        #             with path.open('r', encoding='UTF-8') as infile:
        #                 existing_json = json.load(infile)
        #             existing_meta = existing_json.get('meta')
        #             if count_exhausted(existing_meta):
        #                 print(f'Skipping {author_id} based on {path}')
        #                 break
                
        
        #         if path.exists():
        #             print(f"Skipping {path}")
        #             continue

        sleep_until_one_second_after(last_server_hit)
        print(f'Going back to OpenAlex after {round((dt.datetime.now() - last_server_hit).total_seconds(), 1)} seconds')
        try:            
            last_server_hit = dt.datetime.now()
            print(last_server_hit)
            raw = httpx.get(url, timeout=request_timeout).json()
            last_server_hit = dt.datetime.now()
        except Exception as err:
            error_path.write_text("\n".join((full_author_id, url, str(err))))
            print(err)
            print(error_path)
            continue

        print(raw['meta'])
        
        if raw['results']:
            print(path)
            
            new_result = {}
            new_result['meta'] = raw['meta']
            new_result['results'] = [strip_work(w) for w in raw['results']]
            
            with path.open('w', encoding='UTF-8') as outfile:
                json.dump(new_result, outfile)
               
        # This (or an earlier) page has exhausted the record count of this author
        if count_exhausted(raw['meta']):
            break

283 of 284 A2588028427 19
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2588028427&per_page=20&page=1
Going back to OpenAlex after 1.0 seconds
2022-09-02 18:39:13.136719
{'count': 113, 'db_response_time_ms': 67, 'page': 1, 'per_page': 20}
data\works_by_author\A2588028427-001.json
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2588028427&per_page=20&page=2
Going back to OpenAlex after 1.0 seconds
2022-09-02 18:39:16.195063
{'count': 113, 'db_response_time_ms': 29, 'page': 2, 'per_page': 20}
data\works_by_author\A2588028427-002.json
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2588028427&per_page=20&page=3
Going back to OpenAlex after 1.0 seconds
2022-09-02 18:39:18.760349
{'count': 113, 'db_response_time_ms': 36, 'page': 3, 'per_page': 20}
data\works_by_author\A2588028427-003.json
https://api.openalex.org/works?mailto=matvan@umich.edu&filter=author.id:A2588028427&per_page=20&page=4
Going back to OpenAlex af